item-based sander en hugo

In [7]:
import json
from pandas import Series, Timestamp
import pandas as pd
from pandas import DataFrame
from numpy import nan
import sklearn.metrics.pairwise as pw


In [8]:
def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

In [40]:
data_path = '../data/oregon/'
review_file = 'review.json'
reviews = load_jsons(data_path, review_file)

print(reviews)

                  review_id                 user_id             business_id  \
0    MjA_onNHrXt3XpLpUk8TbA  NkXcAwUXlYoCS8ggzC6rqA  wYgSafshbu0JpKh-LiPisA   
1    3f4pqNIclMMuTxgIkVQ83g  sh9lnj7mCyL01Hde78O9gA  wYgSafshbu0JpKh-LiPisA   
2    K7Z3yZU9uW5-t6moEyRaig  3SgYeDm2aIf6RkWzTGsNCg  xQHoR3VNqBZODz30FwahnA   
3    U-wrXnz3J2_2TGeTCEHIqw  d5WLqmTMvmL7-RmUDVKqqQ  xQHoR3VNqBZODz30FwahnA   
4    4OmqSql5ETH1vkR15IBnQg  ESbzOOA8LkipP4FT1iaXbw  MIk9D2NemI-F4AeopYrTzw   
..                      ...                     ...                     ...   
853  AiUwBtmlbXPQB7DcayiK_A  rubuMDJC7K-5SH4ST2ygfg  Jdtyt2bHpo14dQX5SF_hXQ   
854  JJrA3LNNUqcQdsypONx19A  B2bv64N4vXoWjhlpGYFzbw  Syy4Gr1JrxICbiDm64go3Q   
855  loozPuBZquuh9F-Ryu4jxg  SdJE4TJ1R8rbA3XfFhUDqg  Syy4Gr1JrxICbiDm64go3Q   
856  xNaOeLPLTZsyvM_Bv-_Urw  xHzZOs8vw7XczppeVWVigQ  pr9GzyrtkWLt67sKYOhXYg   
857  9rGf1RJUCcUJAw1qyx7j0w  pV-M_J6kCjz0pPeIrMdYiw  Jdtyt2bHpo14dQX5SF_hXQ   

     stars  useful  funny  cool  \
0      1.0      

In [53]:
display(reviews[reviews["business_id"] == "wYgSafshbu0JpKh-LiPisA"])

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,MjA_onNHrXt3XpLpUk8TbA,NkXcAwUXlYoCS8ggzC6rqA,wYgSafshbu0JpKh-LiPisA,1.0,1,0,0,If they ever called me back Id know the level ...,2017-11-06 17:40:11
1,3f4pqNIclMMuTxgIkVQ83g,sh9lnj7mCyL01Hde78O9gA,wYgSafshbu0JpKh-LiPisA,5.0,1,1,0,Rich and his team of skilled electricians do f...,2011-03-21 16:10:52
12,swGSxqlsjaXxwuR3ofK3_Q,BXYtQvMrYmzwfLEt-OxOcQ,wYgSafshbu0JpKh-LiPisA,5.0,0,0,0,This is my third time using Searl.\n\nI feel l...,2015-01-22 22:10:12
25,384o5_axabZhCmnz0wVAFA,UVXfoZ2x6tpkGREBuBZSzQ,wYgSafshbu0JpKh-LiPisA,5.0,0,0,0,"Searl Electric is fantastic. Our electrician,...",2016-03-26 11:30:55


In [46]:
def pivot_ratings(ratings):
    return df.pivot(values='rating', columns='userId', index='movieId')

In [47]:
pivot_ratings(reviews)

ValueError: Index contains duplicate entries, cannot reshape